In [5]:
import yfinance as y
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt 

Number_simulation=1000000
coupon=0.08
barrier_autocall=1
barrier_coupon=0.6
barrier_capital=0.5
T=5
N=5
r=0.05
t=T/N
tickers=['AAPL','GOOG','TSLA','MSFT']

data=y.download(tickers=tickers,start='2020-01-01',end=dt.datetime.now(),auto_adjust=False)['Adj Close']

S=np.ones(len(tickers))


log_return=np.log(data/data.shift(1))
log_return.dropna(inplace=True)
vol=log_return.std()*np.sqrt(252)
vol=np.reshape(vol,shape=(len(tickers),1))

corr=log_return.corr()
corr
corr=np.linalg.cholesky(corr,upper=False)

Z=np.random.standard_normal(size=(Number_simulation,len(tickers),N))

Z_corr = np.matmul(corr, Z)

drift=(r-0.5*vol**2)*t
noise=vol*np.sqrt(t)*Z_corr

Path=np.zeros(shape=(Number_simulation,len(tickers),N+1))
Path[:,:,0]=S
S=np.reshape(S,shape=(len(tickers),1))
Path[:,:,1:]=S*np.exp(np.cumsum((drift+noise),axis=2))

payoff=np.zeros(Number_simulation)

min_path=np.min(Path,axis=1)

observation=min_path[:,1:]

is_called=(observation>barrier_autocall)
mask_dates=np.any(is_called,axis=1)
dates=np.argmax(is_called,axis=1)
dates=dates[mask_dates]+1
real_dates=dates*t
payoff[mask_dates]=(1+dates*coupon)*np.exp(-r*real_dates)


not_called=~mask_dates
final_prices=observation[not_called,-1]

payout_capital = np.where( final_prices >= barrier_capital, 1,final_prices)

payout_coupon = np.where(final_prices >= barrier_coupon,coupon * T, 0.0)
final_payout = payout_capital + payout_coupon
payoff[not_called] = final_payout*np.exp(-r*T)


np.mean(payoff)

[*********************100%***********************]  4 of 4 completed


np.float64(0.5972296690749177)